<a href="https://colab.research.google.com/github/Shun0212/CodeBERTPretrained/blob/main/UseMyCodeBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [3]:
import os
import random
import torch
import torch.nn.functional as F
from transformers import BertForMaskedLM, BertTokenizerFast
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


# モデルとトークナイザーのロード
model_name = "Shuu12121/CodeMorph-BERT"
print("モデルとトークナイザーをロードします...")
tokenizer = BertTokenizerFast.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

# デバイス設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用デバイス: {device}")
model.to(device)



モデルとトークナイザーをロードします...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/224k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/728k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/254M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


generation_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

使用デバイス: cpu


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [10]:
# =======================
# 2. CodeBERT を用いた MASK 付与・予測関数
# =======================
def mask_and_predict_codebert(text, top_k=5):
    """
    入力コードの一部を [MASK] に置換し、予測結果を出力する関数。
    """
    print("\n=== 予測を開始 ===")
    print(f"入力コード:\n{text}")

    # [CLS] と [SEP] を自動付与してエンコード
    token_ids = tokenizer.encode(text, add_special_tokens=True)
    if len(token_ids) <= 2:
        print("入力テキストが短すぎます:", text)
        return

    # ランダムにマスクする位置を選択
    mask_index = random.randint(1, len(token_ids) - 2)

    # トークン列を取得（デバッグ用）
    original_tokens = tokenizer.convert_ids_to_tokens(token_ids)

    if mask_index >= len(original_tokens):
        print("マスク位置が範囲外です。スキップします。")
        return

    correct_token = original_tokens[mask_index]  # 正解トークン
    print(f"\nマスクするトークン: {correct_token} (位置: {mask_index})")

    # 指定位置を [MASK] に置換
    token_ids[mask_index] = tokenizer.convert_tokens_to_ids("[MASK]")

    # トークン列を再構成
    masked_tokens = [
        "**[MASK]**" if idx == mask_index else token
        for idx, token in enumerate(tokenizer.convert_ids_to_tokens(token_ids))
    ]
    masked_text = " ".join(masked_tokens)
    print(f"\nマスク後のトークン列:\n{masked_text}")

    # テンソル化
    input_tensor = torch.tensor([token_ids], dtype=torch.long).to(device)

    # 推論
    model.eval()
    with torch.no_grad():
        outputs = model(input_tensor)
        logits = outputs.logits

    # マスク位置のロジットから上位 top_k 件を取得
    mask_logits = logits[0, mask_index]
    top = torch.topk(F.softmax(mask_logits, dim=-1), top_k)

    # 結果出力
    print("\n=== 予測結果 ===")
    print(f"正解トークン: {correct_token}")
    print("予測された単語:")
    for i in range(top_k):
        pred_token = tokenizer.convert_ids_to_tokens(top.indices[i].item())
        prob = top.values[i].item()
        correct_mark = " ✅" if pred_token == correct_token else ""
        print(f"  {i+1}. {pred_token} ({prob:.2%}){correct_mark}")

# テスト用のコードサンプル
sample_texts = [
    "def multiply(a, b):\n    return a * b",
    "if x > 10:\n    print('x is greater than 10')",
    "for i in range(5):\n    print(i)",
    "class Person:\n    def __init__(self, name):\n        self.name = name",
    "try:\n    risky_operation()\nexcept Exception as e:\n    print(e)"
]

print("\n=== テスト対象のコードサンプル ===")
for i, text in enumerate(sample_texts):
    print(f"\n--- サンプル {i+1}/{len(sample_texts)} ---")
    mask_and_predict_codebert(text, top_k=5)


=== テスト対象のコードサンプル ===

--- サンプル 1/5 ---

=== 予測を開始 ===
入力コード:
def multiply(a, b):
    return a * b

マスクするトークン: return (位置: 9)

マスク後のトークン列:
[CLS] def multiply ( a , b ) : **[MASK]** a * b [SEP]

=== 予測結果 ===
正解トークン: return
予測された単語:
  1. return (98.61%) ✅
  2. yield (0.91%)
  3. pass (0.08%)
  4. * (0.07%)
  5. - (0.04%)

--- サンプル 2/5 ---

=== 予測を開始 ===
入力コード:
if x > 10:
    print('x is greater than 10')

マスクするトークン: ' (位置: 8)

マスク後のトークン列:
[CLS] if x > 10 : print ( **[MASK]** x is greater than 10 ' ) [SEP]

=== 予測結果 ===
正解トークン: '
予測された単語:
  1. ' (99.98%) ✅
  2. u (0.00%)
  3. " (0.00%)
  4. : (0.00%)
  5. ) (0.00%)

--- サンプル 3/5 ---

=== 予測を開始 ===
入力コード:
for i in range(5):
    print(i)

マスクするトークン: i (位置: 11)

マスク後のトークン列:
[CLS] for i in range ( 5 ) : print ( **[MASK]** ) [SEP]

=== 予測結果 ===
正解トークン: i
予測された単語:
  1. i (94.60%) ✅
  2. 1 (0.91%)
  3. 0 (0.91%)
  4. 5 (0.35%)
  5. 4 (0.30%)

--- サンプル 4/5 ---

=== 予測を開始 ===
入力コード:
class Person:
    def __init__(self, name):
        self.name = 

In [13]:
# =======================
# 3. CodeSearchNet から test データを読み込み、モデルを適用
# =======================
def test_on_codesearchnet(num_samples=5):
    """
    CodeSearchNet の test データからコードを取得し、マスク補完を試す。
    512トークンを超えるコードはスキップする。
    """
    print("\n=== CodeSearchNet の test データを使用 ===")
    dataset = load_dataset("code_search_net", "python", split="test")

    valid_samples = []
    attempts = 0
    while len(valid_samples) < num_samples and attempts < num_samples * 3:  # 余裕を持って3倍試行
        attempts += 1
        sample = random.choice(dataset)  # ランダムに1つ選択
        code_sample = sample["func_code_string"]

        # トークン数を取得
        tokens = tokenizer.encode(code_sample, add_special_tokens=True)
        if len(tokens) > 512:
            print(f"⚠️ スキップ: {len(tokens)} トークン (512超過)")
            continue  # スキップして次へ

        valid_samples.append(code_sample)

    if not valid_samples:
        print("⚠️ 有効なサンプルが見つかりませんでした。")
        return

    # 選択したサンプルでマスク予測
    for i, code_sample in enumerate(valid_samples):
        print(f"\n=== サンプル {i+1}/{len(valid_samples)} ===\n{code_sample[:300]}...")  # 長すぎる場合は省略
        mask_and_predict_codebert(code_sample)


# 5-2: CodeSearchNet のデータでテスト
test_on_codesearchnet(num_samples=3)


=== CodeSearchNet の test データを使用 ===

=== サンプル 1/3 ===
def _get_separator(num, sep_title, sep_character, sep_length):
    """Get a row separator for row *num*."""
    left_divider_length = right_divider_length = sep_length
    if isinstance(sep_length, tuple):
        left_divider_length, right_divider_length = sep_length
    left_divider = sep_characte...

=== 予測を開始 ===
入力コード:
def _get_separator(num, sep_title, sep_character, sep_length):
    """Get a row separator for row *num*."""
    left_divider_length = right_divider_length = sep_length
    if isinstance(sep_length, tuple):
        left_divider_length, right_divider_length = sep_length
    left_divider = sep_character * left_divider_length
    right_divider = sep_character * right_divider_length
    title = sep_title.format(n=num + 1)

    return "{left_divider}[ {title} ]{right_divider}\n".format(
        left_divider=left_divider, right_divider=right_divider, title=title)

マスクするトークン: _ (位置: 154)

マスク後のトークン列:
[CLS] def _ get _ s

In [14]:
# =======================
# 4. 2つのコードサンプルの類似性を計算
# =======================
def compute_code_similarity(code1, code2):
    """
    2つのコードをトークナイズし、埋め込みベクトルを計算して類似度を算出する。
    """
    print("\n=== コードの類似度を計算 ===")

    # トークナイズして埋め込みベクトルを取得
    tokens1 = tokenizer(code1, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    tokens2 = tokenizer(code2, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

    # ベクトル化
    with torch.no_grad():
        embed1 = model.bert(**tokens1).last_hidden_state.mean(dim=1).cpu().numpy()
        embed2 = model.bert(**tokens2).last_hidden_state.mean(dim=1).cpu().numpy()

    # コサイン類似度を計算
    similarity = cosine_similarity(embed1, embed2)[0][0]

    print(f"コード1:\n{code1[:300]}...\n")
    print(f"コード2:\n{code2[:300]}...\n")
    print(f"類似度スコア: {similarity:.4f}")

# 5-3: 類似度計算のテスト
code_a = "def add_numbers(a, b): return a + b"
code_b = "def sum_values(x, y): return x + y"

compute_code_similarity(code_a, code_b)



=== コードの類似度を計算 ===
コード1:
def add_numbers(a, b): return a + b...

コード2:
def sum_values(x, y): return x + y...

類似度スコア: 0.7653


In [15]:
import pandas as pd


def get_codesearchnet_samples(num_samples=5):
    """
    CodeSearchNet の test データから指定数のサンプルコードを取得する。
    """
    print("\n=== CodeSearchNet の test データを取得 ===")
    dataset = load_dataset("code_search_net", "python", split="test")

    # ランダムに num_samples 件を抽出
    sampled_data = random.sample(list(dataset), num_samples)

    # コードのリストを作成
    code_samples = [sample["func_code_string"] for sample in sampled_data]

    print(f"取得したサンプル数: {len(code_samples)}")
    for i, code in enumerate(code_samples):
        print(f"\n--- サンプル {i+1}/{num_samples} ---")
        print(code[:300], "...")  # 長すぎる場合は最初の300文字のみ表示

    return code_samples


def compute_code_similarity_matrix(code_samples):
    """
    CodeSearchNet のコードサンプル間の類似度を計算し、類似度行列を出力する。
    """
    print("\n=== コードの類似度行列を計算 ===")

    # 各コードをトークナイズし、埋め込みベクトルを取得
    embeddings = []
    for i, code in enumerate(code_samples):
        tokens = tokenizer(code, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

        with torch.no_grad():
            embed = model.bert(**tokens.to(device)).last_hidden_state.mean(dim=1).cpu().numpy()

        embeddings.append(embed)

        print(f"埋め込み取得完了: サンプル {i+1}/{len(code_samples)}")

    # 埋め込みを NumPy 配列に変換
    embeddings = np.vstack(embeddings)

    # コサイン類似度を計算
    similarity_matrix = cosine_similarity(embeddings)

    # 類似度行列を DataFrame 形式で表示
    df = pd.DataFrame(similarity_matrix, columns=[f"Code {i+1}" for i in range(len(code_samples))],
                      index=[f"Code {i+1}" for i in range(len(code_samples))])

    print("\n=== 類似度行列 ===")
    print(df)

    return df


# 5-2: CodeSearchNet のデータを取得
code_samples = get_codesearchnet_samples(num_samples=5)

# 5-3: 類似度を計算
similarity_matrix_df = compute_code_similarity_matrix(code_samples)



=== CodeSearchNet の test データを取得 ===
取得したサンプル数: 5

--- サンプル 1/5 ---
def get_by_receiver(cls, request_id, user):
        """Get access request for a specific receiver."""
        return cls.query.filter_by(
            id=request_id,
            receiver_user_id=user.id
        ).first() ...

--- サンプル 2/5 ---
def equal(self, cwd):
        """ Returns True if left and right are equal
        """
        cmd = ["diff"]
        cmd.append("-q")
        cmd.append(self.left.get_name())
        cmd.append(self.right.get_name())

        try:
            Process(cmd).run(cwd=cwd, suppress_output=True)
         ...

--- サンプル 3/5 ---
def despike(self, expdecay_despiker=True, exponent=None,
                noise_despiker=True, win=3, nlim=12., maxiter=3):
        """
        Applies expdecay_despiker and noise_despiker to data.

        Parameters
        ----------
        expdecay_despiker : bool
            Whether or not to a ...

--- サンプル 4/5 ---
def list_dirnames_in_directory(self, dirname